In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten, GlobalAveragePooling2D
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
image_size = [224, 224]
batch_size = 50

In [ ]:
vgg = VGG16(input_shape=image_size + [3], weights='imagenet', include_top=False)

for layer in vgg.layers[0:-2]:
  layer.trainable = False

58900480/58889256 [==============================] - 0s 0us/step


In [ ]:
folders = glob('/content/drive/MyDrive/standford_dt/car_train/car_train/*')
  
x = Flatten()(vgg.output)
prediction = Dense(len(folders), activation='softmax')(x)
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
#NE
x = Flatten()(vgg.output)
prediction = Dense(157, activation='softmax')(x)
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,shear_range = 0.2,zoom_range = 0.2,horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/standford_dt/car_train/car_train',
                                                 seed = 42,
                                                 target_size = image_size,
                                                 batch_size = 128,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/standford_dt/car_test/cars_test',
                                            seed = 42,
                                            target_size = image_size,
                                            batch_size = 128,
                                            class_mode = 'categorical')

Found 12948 images belonging to 157 classes.
Found 3237 images belonging to 157 classes.


In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss',mode = 'min')

In [ ]:
head_model = model.fit_generator(training_set,validation_data=test_set,epochs=25,steps_per_epoch=len(training_set),validation_steps=len(test_set), verbose = 1 ,callbacks=[callback])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


Epoch 1/25
102/102 [==============================] - 5717s 56s/step - loss: 4.1427 - accuracy: 0.1311 - val_loss: 3.0296 - val_accuracy: 0.2777
Epoch 2/25
102/102 [==============================] - 292s 3s/step - loss: 2.2949 - accuracy: 0.4280 - val_loss: 2.3187 - val_accuracy: 0.4331


In [ ]:
model.save('/content/drive/MyDrive/standford_dt/vgg16')

In [ ]:
# loss for r
plt.plot(head_model.history['loss'], label='train loss')
plt.plot(head_model.history['val_loss'], label='test loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# accuracies for r
plt.plot(head_model.history['accuracy'], label='train accuracy')
plt.plot(head_model.history['val_accuracy'], label='test accuracy')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
#Save model

import tensorflow as tf

from keras.models import load_model

model.save('/content/drive/MyDrive/standford_dt/vgg16')

In [ ]:
#from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image